In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import re
import pandas as pd

In [2]:
#routard
def scrap_routard(continent, country, city):
    """continent, country, city in FRENCH"""
    response = requests.get(f'https://www.routard.com/fr/guide/{continent}/{country}/{city}')
    routard_soup = BeautifulSoup(response.text, 'html.parser')
    links = routard_soup.find_all('a', attrs={'href': re.compile(f'guide/{continent}/{country}/{city}')})
    for link in links:
        print(link.get('href').split("/")[-1])
    # scraper les liens de chaque truc pour récup la description rapide ?

scrap_routard('europe', 'grece', 'athenes')

acropole
agora
erechtheion
jardin-national
mont-lycabette
musee-archeologique-national
musee-de-l-acropole
parlement-place-syndagma
parthenon
quartier-de-plaka


In [2]:
#guide michelin
region = 'attica'
city = 'athens'

In [9]:
COSTS = ['Affordable', 'Mid-Range', 'Premium', 'Luxury']

def scrap_michelin(city, region=None, n_pages=1):
    # works if n_pages > number of pages on the website
    if region is None:
        base_url = f"https://guide.michelin.com/fr/fr/{city}-region/restaurants"
    else:
        base_url = f"https://guide.michelin.com/fr/fr/{region}/{city}/restaurants"
    assert(n_pages>0), "n_pages below 0 !"
    data = []
    for i in range(n_pages):
        if i>0:
            url = base_url + f"/page/{i+1}?sort=distance"
        else:
            url = base_url + "?sort=distance"
        response = requests.get(url)
        michelin_soup = BeautifulSoup(response.text, 'html.parser')
        restaurants = michelin_soup.find_all('div', {'class': "card__menu-content"})
        if len(restaurants)==0:
            break
        for resto in restaurants:
            resto_a = resto.find('a')
            name = resto_a.text.strip()
            link = "https://guide.michelin.com" + resto_a.get('href')
            nb_stars = 0
            bib_gourmand = False #bib-gourmand: nos meilleurs rapports qualité-prix
            gastronomie_durable = False
            gastronomie_durable = False
            for span in resto.find_all('span', {'class':'distinction-icon'}):
                for distinction in span.find_all('img'):
                    src = distinction.get('src')
                    if "star" in src:
                        nb_stars += 1
                    elif "bib-gourmand" in src:
                        bib_gourmand = True
                    elif "gastronomie-durable" in src:
                        gastronomie_durable = True
    
            resto_div = resto.find_all('div', {'class': "card__menu-footer--score"})
            tmp = re.sub(r'\s+', ' ', resto_div[1].text).strip() # to get "<$/€> · <cuisine>"
            price, cuisine = tmp.split(" · ")
            cost = COSTS[len(price)-1]
            address, latitude, longitude = scrap_resto_link(link)
            data.append({
                "name": name,
                "link": link,
                "stars": nb_stars,
                "bib-gourmand": bib_gourmand,
                "cuisine": cuisine,
                "cost": cost,
                "address": address,
                "latitude":latitude,
                "longitude": longitude,
                "gastonomie_durable": gastronomie_durable
            })
    if len(data)==0:
        return None
    return pd.DataFrame(data)

def scrap_resto_link(url):
    resto_req = requests.get(url)
    resto_soup = BeautifulSoup(resto_req.text, 'html.parser')
    address = ""
    for div in resto_soup.find_all('div', {'class':'data-sheet__block--text'}):
        text = div.text.strip()
        if not '\n' in text:
            address = text
            break
    latitude, longitude = None, None
    for iframe in resto_soup.find_all('iframe'):
        i_src = iframe.get('src')
        if i_src is None:
            continue
        if "maps" in i_src:
            search = re.search(r'q=([-+]?[0-9]*\.?[0-9]+),([-+]?[0-9]*\.?[0-9]+)', i_src)
            if search:
                latitude = float(search.group(1))
                longitude = float(search.group(2))
    return address, latitude, longitude

# print(len(scrap_michelin('athens', 'attica')))
# print(len(scrap_michelin('athens', 'attica', 2)))

In [8]:
print(len(scrap_michelin('athens', 'attica', 3)))

36


In [17]:
# https://guide.michelin.com/fr/fr/osaka-region/restaurants
region = "osaka"
city = "osaka"
data = scrap_michelin(city, region, 2)
if data is None:
    data = scrap_michelin(region, 2)
data

In [14]:
# pattern urls:
#      same name region/city:
#      .../osaka-region/osaka/...
#      .../tokyo-region/tokyo/...
#      .../new-york-state/new-york/...
#      .../shanghai-municipality/shanghai/...
#      ...

#      region/city
#      .../attica/athens/...

#      france (region/city):
#      .../grand-est/reims/...

#      same city in us (idk other countries) (can't find index):
#      .../colorado/denver_1261491/...
#      .../denver_2892491_noindex/...
#      .../pennsylvania/denver_2942583_noindex/...

region = "osaka"
city = "osaka"

# possible implementation if you are known to be a fdp
data = scrap_michelin(city, region)
if data is None:
    data = scrap_michelin(region)
if data is None:
    data = scrap_michelin(city, f'{city}-region')
if data is None:
    data = scrap_michelin(city, f'{city}-state')
data

In [12]:
# pour ne pas montrer un écran noir mdt 5min avant que ça retourne quoi que ce soit pdt la prez
from tqdm import tqdm

for i in tqdm(range(10000000)):
    i+1

100%|█████████████████████████████████████████████████████████████████| 10000000/10000000 [00:03<00:00, 2983587.69it/s]


In [ ]:
# pour scrap plusieurs trucs en mm temps plutôt que de devoir attendre la fin d'un scrap pour en commencer un 2e
from threading import Thread

## Suppr stop words 13 langues

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect
from nltk.corpus import stopwords
nltk.download('stopwords')

language_dict = {
  'cs': 'czech',
  'da': 'danish',
  'nl': 'dutch',
  'en':'english',
  'et': 'estonian',
  'fi': 'finnish',
  'fr': 'french',
  'de': 'german',
  'el': 'greek',
  'it': 'italian',
  'no': 'norwegian',
  'pl': 'polish',
  'pt': 'portuguese',
  'ru': 'russian',
  'sl': 'slovene',
  'es': 'spanish',
  'sv': 'swedish',
  'tr': 'turkish'
}


def tokenize_without_stopwords(text):
  try:
    language = language_dict[detect(text)]
    stop_words = set(stopwords.words(language))
    filtered_text = " ".join([word for word in text.split() if word not in stop_words])
    return word_tokenize(filtered_text, language=language)

  except Exception as e:
    if len(str(e)) == 4:
      print("[WARNING] language not supported:", str(e))
    else:
      print("error", e)
    return word_tokenize(text)